In [1]:
import utils

import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
data_dir = 'input'
utils.download_extract('celeba', data_dir)

Extracting celeba...


In [5]:
def discriminator(images, alpha = 0.2, dropout = 0.8):
    """
    Create the discriminator network
    :param images: Tensor of input image(s)
    :param alpha: The leakage amount from the relus.
    :param dropout: How much dropout we want for trainning the discriminator
    :return: Tuple of (tensor output of the discriminator, tensor logits of the discriminator)
    """
    
    x = tf.keras.layers.Conv2D(64, 5, strides = 2, padding = 'same')(images)
    h1 = tf.keras.layers.maximum([alpha * x, x])

    h2 = tf.keras.layers.Conv2D(128, 5, strides = 2, padding = 'same')(h1)
    h2 = tf.keras.layers.BatchNormalization(training = True)(h2)
    h2 = tf.keras.layers.maximum([alpha * h2, h2])

    h3 = tf.keras.layers.Conv2D(256, 5, strides = 2, padding = 'same')(h2)
    h3 = tf.keras.layers.BatchNormalization(training = True)(h3)
    h3 = tf.keras.layers.maximum([alpha * h3, h3])

    flat = tf.keras.layers.Reshape((-1, 4*4*256))(h3)
    dropout_layer = tf.keras.layers.Dropout(dropout)(flat)
    logits = tf.keras.layers.Dense(1)(dropout_layer)
    out = tf.keras.activations.sigmoid(logits)
        
    return tf.keras.models.Model(x , [out, logits])

In [ ]:
def generator(z, out_channel_dim, is_train=True, alpha=0.2):
    """
    Create the generator network
    :param z: Input z
    :param out_channel_dim: The number of channels in the output image
    :param is_train: Boolean if generator is being used for training
    :param alpha: The leakage amount from the relus.
    :return: The tensor output of the generator
    """
    
    x = tf.keras.layers.Dense(3*3*512)(z)
    h1 = tf.keras.layers.Reshape((-1, 3, 3, 512))(x)
    h1 = tf.keras.layers.BatchNormalization(training = is_train)(h1)
    h1 = tf.keras.layers.maximum([alpha * h1, h1])
    # 3x3x512

    h2 = tf.keras.layers.Conv2DTranspose(256, kernel_size = 4, strides = 2, padding = 'same')(h2)
    h2 = tf.keras.layers.BatchNormalization(training = is_train)(h2)
    h2 = tf.keras.layers.maximum([alpha * h2, h2])
    # 6x6x256

    h3 = tf.layers.tf.keras.layers.Conv2DTranspose(h2, 128, kernel_size = 4, strides = 2, padding = 'valid')
    h3 = tf.keras.layers.BatchNormalization(training = is_train)(h3)
    h3 = tf.keras.layers.maximum([alpha * h3, h3])
    #14x14x128

    # Output layer
    logits = tf.keras.layers.Conv2DTranspose(h3, out_channel_dim, kernel_size = 5,strides = 2, padding = 'same')
    out = tf.keras.layers.activation.tanh(logits)
    # 28x28x5
        
    return tf.keras.models.Model(x, out)